# Tests for Magnetic moment

- Useful imports

In [1]:
import os
import scipy.io as sio
from scipy.io import readsav
from scipy.optimize import curve_fit
import numpy as np
from math import *
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

%matplotlib qt

from star import Star
from planet import Planet
from dynamo_region import DynamoRegion
from magnetic_moment import MagneticMoment
from target_selection import Config

- Useful constants :

In [2]:
MS = 1.989e30  # kg
RS = 6.96342e8  # m
AS = 4.6  # yo
BSsw = 1  # T
LS = 3.826e26  # W

MJ = 1.8986e27  # kg
RJ = 69911e3  # m
wJ = 1.77e-4  # s-1
LJ_norm=1.31e-9
dJ=5.2*149597870700

- Configuration and dataset :

In [3]:
config_param = Config()
data =pd.read_csv(r'exoplanet.eu_catalog.csv')
data = config_param.param_names(data=data)

- Instanciating Jupiter and the Sun :

In [4]:
jup = Planet(
    name="Jupiter",
    mass=1.0,
    radius={"models": config_param.planet_radius_models, "radius": 1.0},
    distance=5.2,
    worb={"star_mass": MS, "worb": 1.0},
    luminosity={
        "models": config_param.planet_luminosity_models,
        "luminosity": np.nan,
        "star_age": 4.6,
    },
    wrot=1.0,
)

dyn_region_jup = DynamoRegion.from_planet(planet=jup, rhocrit=config_param.rho_crit)
dyn_region_jup.magnetic_field(planet=jup,rc_dyn=config_param.rc_dyn, jup=True)

sun = Star(
    name="Soleil",
    mass=1.0,
    radius={"models": config_param.star_radius_models, "radius": 1.0},
    age=AS,
    obs_dist=1.0,
    sp_type='V'
)
print(sun.luminosity)

0.6977165691451518


### Test plot of B(t) from Reiners&Christensen's paper

In [5]:
table_1MJ=pd.read_csv(r'/Users/emauduit/Documents/These/target_selection/Programmes/1MJ.csv', delimiter=';')
table_5MJ=pd.read_csv(r'/Users/emauduit/Documents/These/target_selection/Programmes/5MJ.csv', delimiter=';')
table_10MJ=pd.read_csv(r'/Users/emauduit/Documents/These/target_selection/Programmes/10MJ.csv', delimiter=';')

In [6]:
B_1=[]

for i in range(20):
    L=table_1MJ['L_Ls'][i]
    R=table_1MJ['R_m'][i]/sun.radius
    Mp=1*jup.mass/sun.mass
    a=Mp*pow(L,2)*pow(R,-7)
    b=pow(1-(0.17/(1.0)),3)
    res=4.8*b*pow(a,1/6)/(2*np.sqrt(2))
    B_1.append(2*res*1e3)

B_5=[]

for i in range(20):
    L=table_5MJ['L_Ls'][i]
    R=table_5MJ['R_m'][i]/sun.radius
    Mp=5*jup.mass/sun.mass
    a=Mp*(L**2)*pow(R,-7)
    b=pow(1-(0.17/(5.0)),3)
    res=4.8*b*pow(a,1/6)/(2*sqrt(2))
    B_5.append(2*res*1e3)

B_10=[]

for i in range(20):
    L=table_10MJ['L_Ls'][i]
    R=table_10MJ['R_m'][i]/sun.radius
    Mp=10*jup.mass/sun.mass
    a=Mp*(L**2)*pow(R,-7)
    b=pow(1-(0.17/(10.0)),3)
    res=4.8*b*pow(a,1/6)/(2*sqrt(2))
    B_10.append(2*res*1e3)

In [7]:
plt.figure()
plt.plot(table_1MJ['t_Gyr']*1e9,B_1,'+', label='1 $M_J$')
plt.plot(table_5MJ['t_Gyr']*1e9,B_5,'+', label='5 $M_J$')
plt.plot(table_10MJ['t_Gyr']*1e9,B_10,'+', label='10 $M_J$')
plt.xlabel('Age [Gyr]')
plt.ylabel('B [G]')
plt.title("Evolution de B en fonction de l'âge")
plt.xscale('log')
plt.yscale('log')
plt.grid()
plt.legend()

### Test of values for a small list of targets to compare with JMG thesis

We add this table of luminosities in the model of Reiners&Christensen for the estimation of the dipolar magnetic moment.
The table gives values of L and Rp with respect to the age of the planet. 

Suggestion/idea : we can estimate the age of the planet by saying it is t*-0,5 en Gyr ? --> we then retreive the value of R and L associated. If we know the radius of the planet, we simply take the value of L associated to the closest value of Rp in the table.

Nota : For the two model of Mizutani we suppose that the electric conductivity in the dynamo region is the same as Jupiter, so normalized to Jupiter this value is 1.

In [8]:
test_targets=pd.DataFrame({'star_radius' : [1.0,1.0,1.18,1.12,1.48,1.48,1.48], 
                'star_mass':[1.0,1.0,1.06,1.1,1.42,1.42,1.42],
                'star_age' : [4.6,4.6,5.5,3.0,1.0,1.0,1.0], 
                'planet_radius':[1.0,0.84,1.42,1.25,1.2,1.58,1.48], 
                'planet_mass':[1.0,0.3, 0.69,1.18,4.4,7.0,10.0], 
                'planet_orbital_frequency':[1.0,0.93,0.12,0.34,0.12,0.12,0.12], 
                'planet_distance':[5.2,9.5,0.045,0.0225,0.0489,0.0489,0.0489]}, 
                index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])
#print(test_targets)
#print(test_targets.loc['Jupiter'])

rc1=[]
rc2=[]
rhoc1=[]
rhoc2=[]

rc1_norm=[]
rc2_norm=[]

for x in test_targets.itertuples():
    pla = Planet(
            name=x.Index,
            mass=x.planet_mass,
            radius={"models": config_param.planet_radius_models, "radius": x.planet_radius},
            distance=x.planet_distance,
            worb={"star_mass": x.star_mass, "worb": x.planet_orbital_frequency},
            luminosity={
                "models": config_param.planet_luminosity_models,
                "luminosity": np.nan,
                "star_age": x.star_age,
            },
            detection_method='bla',
            wrot=1.0,
        )

    r1=DynamoRegion._LaneEmden(pla.unnormalize_mass(), pla.unnormalize_radius(),1000)
    r2=DynamoRegion._LaneEmden(pla.unnormalize_mass(), pla.unnormalize_radius(),700)
    rc1.append(r1)
    rc2.append(r2)
    rc1_norm.append(r1/pla.unnormalize_radius())
    rc2_norm.append(r2/pla.unnormalize_radius())

    rho1=DynamoRegion._rhodyn(pla.unnormalize_mass(),pla.unnormalize_radius(),r1)
    rho2=DynamoRegion._rhodyn(pla.unnormalize_mass(),pla.unnormalize_radius(),r2)
    rhoc1.append(rho1)
    rhoc2.append(rho2)

rc1_normJ=np.array(rc1)/rc1[0]
rc2_normJ=np.array(rc2)/rc2[0]
rhoc1_norm=np.array(rhoc1)/rhoc1[0]
rhoc2_norm=np.array(rhoc2)/rhoc2[0]

test_results_dynamo=pd.DataFrame({'rc1':rc1_norm, 'rc2':rc2_norm, 'rhoc1':rhoc1_norm, 'rhoc2':rhoc2_norm}, index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'] )


print(test_results_dynamo)


                        rc1       rc2     rhoc1     rhoc2
Jupiter            0.791454  0.848782  1.000000  1.000000
Saturn             0.615022  0.721287  0.676447  0.647143
HD 209458b         0.000000  0.438998  0.000000  0.442728
OGLE-TR-56b        0.672431  0.762088  0.742792  0.718751
T-bootes (light)   0.911164  0.936492  1.957362  2.065509
T-bootes (medium)  0.875936  0.910804  1.483899  1.536665
T-bootes (heavy)   0.925786  0.947091  2.285726  2.433248


In [9]:
dyn_region_jup = DynamoRegion.from_planet(planet=jup, rhocrit=config_param.rho_crit)
dyn_region_jup.magnetic_field(planet=jup,rc_dyn=config_param.rc_dyn, jup=True)
mag_moment_jup = MagneticMoment(models=config_param.magnetic_moment_models, Mm=1.56e27, Rs=1.0)

In [10]:
M_sano1=[] ; M_sano2=[]
M_Busse1=[] ; M_Busse2=[]
M_MizuM1=[] ; M_MizuM2=[]
M_MizuS1=[] ; M_MizuS2=[]
M_RC1=[] ; M_RC2=[]

i=0
for x in test_targets.itertuples():
    r1=rc1_normJ[i] ; r2=rc2_normJ[i]
    rho1=rhoc1_norm[i] ; rho2=rhoc2_norm[i]
    
    star = Star(
    name="Name",
    mass=x.star_mass,
    radius={"models": config_param.star_radius_models, "radius": x.star_radius},
    age=x.star_age,
    obs_dist=1.0,
    sp_type='V'
)
    pla = Planet(
            name=x.Index,
            mass=x.planet_mass,
            radius={"models": config_param.planet_radius_models, "radius": x.planet_radius},
            distance=x.planet_distance,
            worb={"star_mass": x.star_mass, "worb": x.planet_orbital_frequency},
            luminosity={
                "models": config_param.planet_luminosity_models,
                "luminosity": np.nan,
                "star_age": x.star_age,
            },
            detection_method='bla',
            wrot=1.0 )
    dyn_region_pla1 = DynamoRegion(rhocrit=config_param.rho_crit, rhoc=rho1, rc=r1)
    dyn_region_pla1.magnetic_field(pla,config_param.rc_dyn)
    dyn_region_pla2 = DynamoRegion(rhocrit=config_param.rho_crit, rhoc=rho2, rc=r2)
    dyn_region_pla2.magnetic_field(pla,config_param.rc_dyn)
    M_sano1.append(MagneticMoment._Sano(r1,pla.rotrate,rho1))
    M_sano2.append(MagneticMoment._Sano(r2,pla.rotrate,rho2))
    M_Busse1.append(MagneticMoment._Busse(r1,pla.rotrate,rho1))
    M_Busse2.append(MagneticMoment._Busse(r2,pla.rotrate,rho2))
    M_MizuM1.append(MagneticMoment._Mizu_moderate(r1,pla.rotrate,rho1,1.0))
    M_MizuM2.append(MagneticMoment._Mizu_moderate(r2,pla.rotrate,rho2,1.0))
    M_MizuS1.append(MagneticMoment._Mizu_slow(r1,pla.rotrate,rho1,1.0))
    M_MizuS2.append(MagneticMoment._Mizu_slow(r2,pla.rotrate,rho2,1.0))

    M_RC1.append(MagneticMoment._Reiners_Christensen(pla, jup,dyn_region_pla1,dyn_region_jup))
    M_RC2.append(MagneticMoment._Reiners_Christensen(pla, jup,dyn_region_pla2,dyn_region_jup))
    i+=1

M_sano1_norm=np.array(M_sano1)/M_sano1[0]
M_sano2_norm=np.array(M_sano2)/M_sano2[0]
M_RC1_norm=np.array(M_RC1)/M_RC1[0]
M_RC2_norm=np.array(M_RC2)/M_RC2[0]

test_results_moments=pd.DataFrame({'M_sano1': M_sano1, 'M_sano2' : M_sano2, 'M_RC1' : M_RC1, 'M_RC2' : M_RC2, 'M_Busse1': M_Busse1, 'M_Busse2':M_Busse2, 'M_MizuM1':M_MizuM1, 'M_MizuM2':M_MizuM2, 'M_MizuS1':M_MizuS1,'M_MizuS2':M_MizuS2},index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])

    
#print(test_results_dynamo)
print(test_results_moments) 


                     M_sano1   M_sano2      M_RC1      M_RC2   M_Busse1  \
Jupiter             1.000000  1.000000   1.000000   1.000000   1.000000   
Saturn              0.184809  0.247212   0.041186   0.041186   0.149312   
HD 209458b          0.000000  0.225897   0.952268   0.952268   0.000000   
OGLE-TR-56b         1.063885  1.269712   2.397023   2.397023   1.096379   
T-bootes (light)    4.335784  3.838185  16.405752  16.405752   5.096166   
T-bootes (medium)   8.613202  7.866665  41.376722  41.376722  11.389808   
T-bootes (heavy)   10.321072  9.028061  50.281341  50.281341  13.579946   

                    M_Busse2   M_MizuM1  M_MizuM2  M_MizuS1  M_MizuS2  
Jupiter             1.000000   1.000000  1.000000  1.000000  1.000000  
Saturn              0.208865   0.184809  0.247212  0.228744  0.292600  
HD 209458b          0.193592   0.000000  0.225897  0.000000  0.263593  
OGLE-TR-56b         1.345131   1.063885  1.269712  1.032355  1.198521  
T-bootes (light)    4.416422   4.335784

Plot of magnetic moments values computed with (+) and without (x) taking the tidal locking into account. If planets are tidally locked, their rotation rate is taken equal to their orbital period, and if not it is taken equal to Jupiter's rotation rate.

In [11]:
fig, ax = plt.subplots()
xs=range(7)
labels = test_results_moments.index
def format_fn(tick_val, tick_pos):
    if int(tick_val) in xs:
        return labels[int(tick_val)]
    else:
        return ''

ax.xaxis.set_major_formatter(format_fn)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.plot(xs,test_results_moments['M_sano1'], '+-', color='blue', label='Sano_1')
ax.plot(xs,test_results_moments['M_sano2'], '+-', color='deepskyblue', label='Sano_2')
ax.plot(xs,test_results_moments['M_Busse1'], '+-', color='green', label='Busse_1')
ax.plot(xs,test_results_moments['M_Busse2'], '+-', color='limegreen', label='Busse_2')
ax.plot(xs,test_results_moments['M_MizuM1'], '+-', color='red', label='MizuM_1')
ax.plot(xs,test_results_moments['M_MizuM2'], '+-', color='maroon', label='MizuM_2')
ax.plot(xs,test_results_moments['M_MizuS1'], '+-', color='indigo', label='MizuS_1')
ax.plot(xs,test_results_moments['M_MizuS2'], '+-', color='darkviolet', label='MizuS_2')
plt.legend()
plt.xlabel('Object')
plt.ylabel('Magnetic moment [Mj]')
plt.show()

### Compute magnetic moment for all the possible models separately

In [12]:
dynamo_radius = [] ; data_radius = []
mizu_slow = [] ; mizu_mod = []
rein_chris = [] ; rein_chris_dyn = []
sano = []
busse = []

In [13]:
for target in data.itertuples():
    print(target.name)
    if 'PSR' in target.name :
        print('Warning : {} has been skipped'.format(target.name))
        continue

    if (
        not np.isnan(target.semi_major_axis)
        and (not np.isnan(target.mass) or not np.isnan(target.mass_sini))
        and not np.isnan(target.star_mass)
    ):

        if np.isnan(target.mass):
            planet_mass = target.mass_sini * np.sqrt(4 / 3.0)
        else:
            planet_mass = target.mass
        if not np.isnan(target.eccentricity):
            planet_distance = target.semi_major_axis * (1 - target.eccentricity)
        else:
            planet_distance = target.semi_major_axis

        if np.isnan(target.star_age):
            star_age = 5.2
        else:
            if target.star_age <= 1e-7:
                print("Stellar age is too small (< 100 yr) !!!")
                continue
            elif target.star_age < 0.5:
                star_age = 0.5
            else:
                star_age = target.star_age
        
        planet = Planet(
            name=target.name,
            mass=planet_mass,
            radius={"models": config_param.planet_radius_models, "radius": target.radius},
            distance=planet_distance,
            worb={"star_mass": target.star_mass, "worb": np.nan},
            luminosity={
                "models": config_param.planet_luminosity_models,
                "luminosity": np.nan,
                "star_age": star_age,
            },
            detection_method=target.detection_type,
            wrot=1.0,
        )
        star = Star(
            name="star",
            mass=target.star_mass,
            radius={"models": config_param.star_radius_models, "radius": target.star_radius},
            age=star_age,
            obs_dist=target.star_distance,
            sp_type='V'
        )

        if np.isnan(target.radius) and config_param.radius_expansion :
            planet.radius_expansion(luminosity=star.luminosity, eccentricity=target.eccentricity)

        try:
            planet.tidal_locking(age=star.age, star_mass=star.mass)
        except OverflowError:
            continue
        data_radius.append(target.radius)
        dyn_region = DynamoRegion.from_planet(planet=planet, rhocrit=config_param.rho_crit)
        dyn_region.normalize(other=dyn_region_jup)
        dyn_region.magnetic_field(planet=planet, rc_dyn=False)

        dynamo_radius.append(dyn_region.radius)

        magnetic_moment_busse = MagneticMoment(models=['busse'], Mm=1.0, Rs=1.0)
        magnetic_moment_busse.magnetic_moment(
            dynamo=dyn_region, planet=planet, jup=jup, dynamo_jup = dyn_region_jup
        )
        busse.append(magnetic_moment_busse.mag_moment)

        magnetic_moment_sano = MagneticMoment(models=['sano'], Mm=1.0, Rs=1.0)
        magnetic_moment_sano.magnetic_moment(
            dynamo=dyn_region, planet=planet, jup=jup, dynamo_jup = dyn_region_jup
        )
        sano.append(magnetic_moment_sano.mag_moment)

        magnetic_moment_mizus = MagneticMoment(models=['mizu_slow'], Mm=1.0, Rs=1.0)
        magnetic_moment_mizus.magnetic_moment(
            dynamo=dyn_region, planet=planet, jup=jup, dynamo_jup = dyn_region_jup
        )
        mizu_slow.append(magnetic_moment_mizus.mag_moment)

        magnetic_moment_mizum = MagneticMoment(models=['mizu_mod'], Mm=1.0, Rs=1.0)
        magnetic_moment_mizum.magnetic_moment(
            dynamo=dyn_region, planet=planet, jup=jup, dynamo_jup = dyn_region_jup
        )
        mizu_mod.append(magnetic_moment_mizum.mag_moment)

        magnetic_moment_RC = MagneticMoment(models=['rein-chris'], Mm=1.0, Rs=1.0)
        magnetic_moment_RC.magnetic_moment(
            dynamo=dyn_region, planet=planet, jup=jup, dynamo_jup = dyn_region_jup
        )
        rein_chris.append(magnetic_moment_RC.mag_moment)

        dyn_region.magnetic_field(planet=planet, rc_dyn=True)
        magnetic_moment_RCdyn = MagneticMoment(models=['rein-chris-dyn'], Mm=1.0, Rs=1.0)
        magnetic_moment_RCdyn.magnetic_moment(
            dynamo=dyn_region, planet=planet, jup=jup, dynamo_jup = dyn_region_jup
        )
        rein_chris_dyn.append(magnetic_moment_RCdyn.mag_moment)

109 Psc b
112 Psc b
112 Psc c
11 Com b
11 UMi b
14 And b
14 Her b
14 Her c
15 Sge b
16 Cyg B b
18 Del b
1RXS 1609 b
1RXS J103137.1-690205  b
1RXS J125608.8-692652 (AB)b
1RXS J131752.0-505845 b
1RXS J235133.3+312720 b
1SWASP J1407 b
20 LMi Ab


23 Lib b
23 Lib c
24 Boo b
24 LMi b
24 Sex b
24 Sex c
29 Cyg b
2M0045+16
2M 0103-55 (AB) b
2M0117-34
2M 0219-39 b
2M0355
2M0418+21
2M0437 b
2M 0441+23 b
2M0444
2M0838+15 b
2M0838+15 c
2M1059-21 b
2M1207 A
2M1207 b
2M 1252+2735 b
2M 1253+2734 b
2M 1258+2630 b
2M1510 A
2M1510 Ab
2M1510 Ac
2M1510 B
2M 2140+16 b
2M 2206-20 b
2M 2236+4751 b
2MASS 0359+2009 b
2MASS 1534-29 b
2MASS J0001+1535
2MASS J00103250+1715490 b
2MASS J0021-4514 b
2MASS J0030-1450
2MASS J0031+5749
2MASS J00413538-5621127 A
2MASS J00413538-5621127 B
2MASS J01225093-2439505 b
2MASS J0141-4633
2MASS J0153-6744
2MASS J0226-5327
2MASS J0249-0557A
2MASS J0249-0557 (AB)b
2MASS J0249-0557B
2MASS J0323-4631
2MASS J0326-2102
2MASS J0342-6817
2MASS J0348-6022
2MASS J0407+1546
2MASS J0437+2331
2MASS J0443+0002
2MASS J0447-1216
2MASS J0453-1751
2MASS J0459-2853
2MASS J0502+1442
2MASS J0506+5236
2MASS J0518-2756
2MASS J0536-1920
2MASS J05581644-4501559 b
2MASS J0642+4101
2MASS J0718-6415
2MASS J0809+4434
2MASS J0850+1057


2MASS J0850+1057 b
2MASS J0856-1342
2MASS J0951-8023
2MASS J0953-5055 b
2MASS J10475385+2124234
2MASS J11011926-7732383 A
2MASS J11011926-7732383 B
2MASS J1115+1937
2MASS J1119-1137 b
2MASS J1154-3400
2MASS J1155-7919 b
2MASS J1219+3128
2MASS J1247-3816
2MASS J1450-7841 A
2MASS J1450-7841 B
2MASS J15344984-2952274 A
2MASS J15344984-2952274 B
2MASS J1551+0941
2MASS J1554-3154 b
2MASS J155947+440359 b
2MASS J1617-3828 b
2MASS J1647+5632
2MASS J18071593+5015316
2MASS J1935-2846
2MASS J2002-0521
2MASS J2013-2806
2MASS J2117-2940
2MASS J2139+02
2MASS J2206+3301
2MASS J2206-4217
2MASS J2250+2325 b
2MASS J2322-6151
2MASS J2343-3646
2MASS J2352-1100
2MASSW J0829+2655
2MASSW J0928-1603
2MASSW J1326-2729
2MASSW J2208136+292121
2M J2126-81 b
2S 0918-549 b
38 Vir b
3 Cnc b
41 Com b
44 Uma b


47 Uma b
47 Uma c
47 Uma d
4FGL J1408.6-2917 b
4U 1543-62 b
4U 1626-67 b
51 Eri b
51 Peg b
54 Psc b
54 Psc c
55 Cnc Ab
55 Cnc Ac
55 Cnc Ad
55 Cnc Ae
55 Cnc Af
59 Dra b
59 Vir b
61 Vir b
61 Vir c
61 Vir d


6 Lyn b
70 Vir b
75 Cet b
75 Cet c
78 Psc b
79 Cet Ab
81 Cet b
82 Eri b
82 Eri c
82 Eri d
82 Eri e
83 Leo Bb
83 Leo Bc
8 Umi b
94 Cet Ab
AB Aur b
AB Dor C b
AB Pic b
AD 3116 b
AE 1 b
AF Lep b


...

WASP-71 b
WASP-72 b
WASP-73 b
WASP-74 b
WASP-75 b
WASP-76 b
WASP-77 A b
WASP-78 b
WASP-79 b
WASP-7 b
WASP-80 b
WASP-81 b
WASP-81 c


WASP-82 b
WASP-83 b
WASP-84 b
WASP-84 c
WASP-85 A b
WASP-87 A b
WASP-88 b
WASP-89 b
WASP-8 b
WASP-8 c
WASP-90 b
WASP-91 b


WASP-92 b
WASP-93 b
WASP-94 A b
WASP-94 B b
WASP-95 b
WASP-96 b
WASP-97 b
WASP-98 b
WASP-99 b
WD 0137-349 b
WD 0806-661 B b
WD 0837+185 B
WD 1032+011 b
WD 1856+534 b
WD J0601-4625 b
WD J0914+1914 b
WD J2122+6600 b
Wendelstein-1 b
Wendelstein-2 b


WISE0047
WISE 0359-54
WISE 0855-0714
WISE 1118+3125
WISE 1711+3500 b
WISE 2005+5424
WISE 2150A b
WISEA J083011.95+283716.0
WISEA J0919-5748 b
WISEA J2318+0521 b
WISEA J2354-0811
WISE J0323+5625
WISE J0336-0143 b
WISE J041358.14-475039.3
WISE J0458+6434 A
WISE J0458+6434 B
WISE J0528+0901
WISE J0647-6232
WISE J0720-0846
WISE J1217+1626
WISE J1217+1626 b
WISE J1355-8258 b
WISE J1541-2250
WISE J1617+1807
WISE J1639-6847
WISE J1741-4642
WISE J1810-1010
WISE J1812+2721
WISE J1828+2650
WISE J1828+2650 b
WISE J2018-7423
WISE J2216+1952
WISE J2313-8037
WISE J2319+7645
WISE J2359-7335
Wolf 1061 b
Wolf 1061 c
Wolf 1061 d
Wolf 1069 b
Wolf 327 b
Wolf 359 c
Wolf 503 b
Wolf 940 b
WT 5 (AB)b
WTS-1 b
WTS-2 b
XO-1 b
XO-2N b


XO-2 S b
XO-2 S c
XO-3 b
XO-4 b
XO-5 b
XO-6 b
XO-7 b
XTE J0929-314 b
XTE J1751-305 b
XTE J1807-294 b
YBP1194 b
YBP1514 b
YBP401 b
YSES 2 b
YZ Cet b
YZ Cet c
YZ Cet d
zet Del B
ZTF J0003+14 b
ZTF J0038+2030 b
ZTF J0220+21 b
ZTF J0407-00 b
ZTF J1406+1222 Ab
ZTF J1622+47 b
ZTF J1637+49 b
ZTF J2252-05 b


In [14]:
dynamo_radius = np.array(dynamo_radius) ; data_radius = np.array(data_radius)
mask_plot = (~np.isnan(data_radius)) & (dynamo_radius >0) ; n = len(dynamo_radius[mask_plot])

In [15]:
magnetic_moment = np.zeros(shape=(n,6))
busse = np.array(busse) ; magnetic_moment[:,0] = busse[mask_plot]
sano = np.array(sano) ; magnetic_moment[:,1] = sano[mask_plot]
mizu_slow = np.array(mizu_slow) ; magnetic_moment[:,2] = mizu_slow[mask_plot]
mizu_mod = np.array(mizu_mod) ; magnetic_moment[:,3] = mizu_mod[mask_plot]
rein_chris = np.array(rein_chris) ; magnetic_moment[:,4] = rein_chris[mask_plot]
rein_chris_dyn = np.array(rein_chris_dyn) ; magnetic_moment[:,5] = rein_chris_dyn[mask_plot]

In [16]:
plt.figure(figsize=(10,7))
plt.scatter(dynamo_radius[mask_plot], busse[mask_plot], marker='x', color='tab:blue', label = 'Busse')
plt.scatter(dynamo_radius[mask_plot], sano[mask_plot], marker='x', color='tab:orange', label = 'Sano')
plt.scatter(dynamo_radius[mask_plot], mizu_slow[mask_plot], marker='x', color='tab:green', label = 'Mizu slow')
plt.scatter(dynamo_radius[mask_plot], mizu_mod[mask_plot], marker='x', color='tab:red', label = 'Mizu moderate')
plt.yscale('log')
plt.xlabel('$r_c$ [$r_{c,J}$]',fontsize=14)
plt.ylabel(r'$\mathcal{M}$ [$\mathcal{M}_J$]',fontsize=14)
#plt.title('Evolution of magnetic moment for different models with dynamo radius')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/ch3/fig/plot_Mmag_vs_rcdyn.png', dpi=150 )
#plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Magnetic moment/Mmag_vs_rdyn.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

In [17]:
mag_moment_max_msb = np.max(magnetic_moment[:,0:4],axis=1) ; mag_moment_min_msb = np.min(magnetic_moment[:,0:4],axis=1)
mag_moment_geom_mean_msb = np.power(np.prod(magnetic_moment[:,0:4], axis=1), 1 / 4)
data_radius_sorted = np.sort(data_radius[mask_plot]) ; arg_data_radius_sorted = np.argsort(data_radius[mask_plot])

In [18]:
plt.figure(figsize=(10,7))
#plt.scatter(data_radius[mask_plot], busse[mask_plot], marker='x', color='tab:blue', label = 'Busse')
#plt.scatter(data_radius[mask_plot], sano[mask_plot], marker='x', color='tab:orange', label = 'Sano')
#plt.scatter(data_radius[mask_plot], mizu_slow[mask_plot], marker='x', color='tab:green', label = 'Mizu slow')
#plt.scatter(data_radius[mask_plot], mizu_mod[mask_plot], marker='x', color='tab:red', label = 'Mizu moderate')
plt.fill_between(data_radius_sorted,mag_moment_min_msb[arg_data_radius_sorted],mag_moment_max_msb[arg_data_radius_sorted], alpha=0.5, color='gray', zorder=0,label='Range Mmag min - Mmag max')
plt.scatter(data_radius_sorted,mag_moment_geom_mean_msb[np.argsort(data_radius[mask_plot])], marker='+',color='black',zorder=1, label='Geometric mean')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$R_p$ [$R_J$]',fontsize=14)
plt.ylabel(r'$\mathcal{M}$ [$\mathcal{M}_J$]',fontsize=14)
#plt.title('Evolution of magnetic moment for different models with dynamo radius')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/ch3/fig/plot_Mmean_vs_Rp_MSB.png', dpi=150 )
#plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Magnetic moment/Mmag_vs_Rp_error.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

In [19]:
plt.figure(figsize=(10,7))
plt.scatter(dynamo_radius[mask_plot], rein_chris[mask_plot], marker='x', color='tab:blue', label = 'Reiners-Christensen')
plt.scatter(dynamo_radius[mask_plot], rein_chris_dyn[mask_plot], marker='x', color='tab:orange', label = 'Reiners-Christensen with dynamo')
plt.yscale('log')
plt.xlabel('$r_c$ [$r_{c,J}$]',fontsize=14)
plt.ylabel(r'$\mathcal{M}$ [$\mathcal{M}_J$]',fontsize=14)
#plt.title('Evolution of magnetic moment for different models with dynamo radius')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/ch3/fig/plot_Mmag_vs_rcdyn_RC.png', dpi=150 )
#plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Magnetic moment/Mmag_vs_rdyn.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

In [20]:
mag_moment_max_rc = np.max(magnetic_moment[:,4:],axis=1) ; mag_moment_min_rc = np.min(magnetic_moment[:,4:],axis=1)
mag_moment_geom_mean_rc = np.power(np.prod(magnetic_moment[:,4:], axis=1), 1 / 2)

In [24]:
plt.figure(figsize=(10,7))
plt.fill_between(data_radius_sorted,mag_moment_min_rc[arg_data_radius_sorted],mag_moment_max_rc[arg_data_radius_sorted], alpha=0.5, color='gray', zorder=0,label='Range Mmag min - Mmag max')
plt.scatter(data_radius_sorted,mag_moment_geom_mean_rc[np.argsort(data_radius[mask_plot])], marker='+',color='black',zorder=1, label='Geometric mean')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$R_p$ [$R_J$]',fontsize=14)
plt.ylabel(r'$\mathcal{M}$ [$\mathcal{M}_J$]',fontsize=14)
#plt.title('Evolution of magnetic moment for different models with dynamo radius')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/ch3/fig/plot_Mmean_vs_Rp_RC.png', dpi=150)
plt.show()

In [22]:
plt.figure(figsize=(10,7))
plt.scatter(data_radius[mask_plot], busse[mask_plot], marker='x', color='tab:blue', label = 'Busse')
plt.scatter(data_radius[mask_plot], sano[mask_plot], marker='x', color='tab:orange', label = 'Sano')
plt.scatter(data_radius[mask_plot], mizu_slow[mask_plot], marker='x', color='tab:green', label = 'Mizu slow')
plt.scatter(data_radius[mask_plot], mizu_mod[mask_plot], marker='x', color='tab:red', label = 'Mizu moderate')
plt.scatter(data_radius[mask_plot], rein_chris[mask_plot], marker='x', color='tab:purple', label = 'Reiners-Christensen')
plt.scatter(data_radius[mask_plot], rein_chris_dyn[mask_plot], marker='x', color='tab:brown', label = 'Reiners-Christensen with dynamo')
plt.yscale('log')
plt.xlabel('Planetary radius [RJ]')
plt.ylabel('Magnetic moment [MJ]')
plt.title('Evolution of magnetic moment for different models with dynamo radius')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Magnetic moment/Mmag_vs_Rp.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()